## Overview
The FMP (Financial Modeling Prep) LangChain integration provides a seamless way to access financial market data through natural language queries. This integration offers two main components:

- FMPDataToolkit: Creates collections of tools based on natural language queries
- FMPDataTool: A single unified tool that automatically selects and uses the appropriate endpoints

The integration leverages LangChain's semantic search capabilities to match user queries with the most relevant FMP API endpoints, making financial data access more intuitive and efficient.
## Setup

In [ ]:
!pip install -U langchain-fmp-data

In [2]:
import os

# Replace with your actual API keys
os.environ["FMP_API_KEY"] = "your-fmp-api-key"  # pragma: allowlist secret
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"  # pragma: allowlist secret

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

In [3]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Instantiation
There are two main ways to instantiate the FMP LangChain integration:
1. Using FMPDataToolkit

In [ ]:
from langchain_fmp_data import FMPDataToolkit

# Basic instantiation
toolkit = FMPDataToolkit()

# Instantiation with specific query focus
market_toolkit = FMPDataToolkit(query="Get stock market prices and technical indicators",num_results=5)

# Instantiation with custom configuration
custom_toolkit = FMPDataToolkit(
    query="Financial analysis",
    num_results=3,
    similarity_threshold=0.4,
    cache_dir="/custom/cache/path",
)

2. Using FMPDataTool


In [4]:
from langchain_fmp_data import FMPDataTool
from langchain_fmp_data.tools import ResponseFormat

# Basic instantiation
tool = FMPDataTool()

# Advanced instantiation with custom settings
advanced_tool = FMPDataTool(
    max_iterations=50,
    temperature=0.2,
)

## Invocation
The tools can be invoked in several ways:

### Direct Invocation



In [ ]:
# Using FMPDataTool
tool = FMPDataTool()

# Basic query
result = tool.invoke({"query": "What's Apple's current stock price?"})

# Advanced query with specific format
detailed_result = tool.invoke({
    "query": "Compare Tesla and Ford's profit margins",
    "response_format": ResponseFormat.BOTH,
})

### Using with LangChain Agents

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent, AgentExecutor

# Setup
llm = ChatOpenAI(temperature=0)
toolkit = FMPDataToolkit(query="Stock analysis", num_results=3)
tools = toolkit.get_tools()

# Create agent
agent = create_openai_functions_agent(llm, tools)
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Run query
response = agent_executor.invoke({"input": "What's the PE ratio of Microsoft?"})

## Advanced Usage
You can customize the tool's behavior:



In [16]:
# Initialize with custom settings
advanced_tool = FMPDataTool(
    max_iterations=50,  # Increase max iterations for complex queries
    temperature=0.2     # Adjust temperature for more/less focused responses
)

# Example of a complex multi-part analysis
query = """
Analyze Apple's financial health by:
1. Examining current ratios and debt levels
2. Comparing profit margins to industry average
3. Looking at cash flow trends
4. Assessing growth metrics
"""

response = advanced_tool.invoke(
    {"query": query, "response_format": ResponseFormat.BOTH}
)

print("Detailed Financial Analysis:")
print(response)

### API Reference
### FMPDataToolkit
